In [1]:
# Code based on tutorial at:
# http://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

import numpy
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
import csv
filename = "/Users/andreamonacchi/Desktop/funjokes.csv"
raw_text = open(filename)
# skip header
raw_text.next()
reader = csv.DictReader(raw_text,fieldnames=['id','joke'])

# create a dataset from the lowercase version of all jokes
in_ds = []
for row in reader:
    in_ds.append(row['joke'].lower())

In [3]:
# merge all jokes into a text
text = "\n".join(in_ds)
rtext = " ".join(in_ds)

In [4]:
# create mapping of unique chars to integers
chars = sorted(list(set(text)))
print chars
char_to_int = dict((c, i) for i, c in enumerate(chars))
print char_to_int

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '}', '~']
{'\n': 0, '!': 2, ' ': 1, '#': 4, '"': 3, '%': 6, '$': 5, "'": 8, '&': 7, ')': 10, '(': 9, '+': 12, '*': 11, '-': 13, '/': 15, '.': 14, '1': 17, '0': 16, '3': 19, '2': 18, '5': 21, '4': 20, '7': 23, '6': 22, '9': 25, '8': 24, ';': 27, ':': 26, '=': 29, '<': 28, '?': 31, '>': 30, '@': 32, '[': 33, ']': 34, '_': 35, 'a': 37, '`': 36, 'c': 39, 'b': 38, 'e': 41, 'd': 40, 'g': 43, 'f': 42, 'i': 45, 'h': 44, 'k': 47, 'j': 46, 'm': 49, 'l': 48, 'o': 51, 'n': 50, 'q': 53, 'p': 52, 's': 55, 'r': 54, 'u': 57, 't': 56, 'w': 59, 'v': 58, 'y': 61, 'x': 60, 'z': 62, '}': 63, '~': 64}


In [5]:
# summarize the dataset
n_chars = len(text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab

Total Characters:  1944200
Total Vocab:  65


In [6]:
dataX = []
dataY = []

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
for i in range(0, n_chars - seq_length, 1):
    seq_in = rtext[i:i + seq_length]
    seq_out = rtext[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

Total Patterns:  1944100


In [7]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [10]:
from keras.layers import recurrent
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
#model.add(LSTM(units=256, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0))
model.add(Dropout(0.2))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
#model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])


In [ ]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, nb_epoch=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
 119424/1944100 [>.............................] - ETA: 13099s - loss: 2.9160